# BrainQuant3D Setup
## 0.1. Download necessary applications/packages as described in the README.md
BrainQuant3D setup for the Data Analysis Workshop May 2024

You may need to download these programs onto your computer:
1. git (for setting up BrainQuant3D as a package): https://git-scm.com/downloads
2. anaconda (for isolating packages in its own Python environment): https://www.anaconda.com/download

## 0.2. Set up the Python Environment
Because BrainQuant3D requires very specific versions of packages, we need to create an individual python environment to keep the necessary versions contained. 

Open your command line (mac, linux) or WSL (windows) and execute these commands:

```
cd <path to directory where this notebook and the .yml file is in> #e.g. cd mnt/ssd or cd ~/Downloads
conda env create -f bq3d_env.yml
conda activate bq3d
pip install git+https://github.com/MehwishUBC/BrainQuant3D_cFos.git
```
> NOTE: In WSL, you may need to specify the disk that your Windows files are running on. For example, the address might be `mnt/c/Users/<your username>/Downloads` for the downloads folder on your C: drive. 

To activate Jupyter Notebooks, run this in your command line/anaconda prompt:
```
jupyter notebook
```
Start with Jupyter Notebook 00_bq3d_setup.ipynb. Choose the newly created "bq3d" conda environment as the kernel to run these Jupyter Notebooks and code. 

> HINT: 
> 
> `pwd` to see the current working directory that you are in.
> 
> `ls` to see the files on your current directory
> 
> `cd <path/folder to navigate to from your present working directory>` to navigate into a directory
> 
> `cd ..` to step out of a directory/go to the parent directory

## 1. Ensure that the Jupyter Notebook kernel has been changed to the "bq3d" environment!
You can modify this in the Kernel > Change Kernel option in the menu bar.

Run the code below to ensure you can import the packages. There may be warnings from bq3d but we will update the configuration files in this Jupyter Notebook so they will be resolved.

In [1]:
#run this code block to ensure everything has been imported correctly.
import bq3d

import os
from pathlib import Path
import yaml
import bq3d
import glob
from pprint import pprint

## 2. Download Allen Brain Atlas datasets
Download Warping.zip, which contains the Allen Brain Atlas datasets (477.9 MB zipped, 2.32 GB unzipped): https://osf.io/wyxa5/?view_only=f0cc945410554286b78c10e41ab511bb

The Allen Brain Atlas’ includes one hemi in the following orientation: 

![Example of a hemisphere brain slice from the Allen Brain Atlas](allen_brain_atlas_hemi.png)

Sagittal view with rostral (olfactory bulb) on top, caudal (cerebellum) on bottom, dorsal on left and ventral on right.

Since the atlas is always oriented this way, we must align our data to match (edit done to parameter.py file in the 01_bq3d_tutorial notebook).

Unzip and move the downloaded Warping directory to a location you will remember (e.g. SSD drive). You may need to move the Warping folder to a location with enough storage before unzipping it.
```
# mv Warping /mnt/ssd
unzip Warping.zip
mv Warping /mnt/ssd
```
Change `brain_atlas_directory` below to the path of the Warping directory that you unzipped. Run the code cell below to ensure that you have the right path.

In [2]:
# On Windows, write paths "D:\brainquant_RC\bq3d_demo\Warping" like "/mnt/d/brainquant_RC/bq3d_demo/Warping"
brain_atlas_directory = "/Users/ytcao/Documents/Warping" #!!!NOTE: modify the path to the downloaded brain atlas directory on your computer

assert any(Path(brain_atlas_directory).iterdir()), "No files found on the brain atlas directory path. Make sure that the path is correct!"

## 3. Navigate to the parameter_template.py file, and update these variables:

In [3]:
print(f"\nreplace this variable in parameter_template.py (line 34):\n\nPathReg = str(Path('{brain_atlas_directory}'))")


replace this variable in parameter_template.py (line 34):

PathReg = str(Path('/Users/ytcao/Documents/Warping'))


## 4. Download the .ilp file and move into the "filters" folder in this directory
Download the .ilp file (2.75 GB unzipped): https://osf.io/jckgr/?view_only=f0cc945410554286b78c10e41ab511bb

Click the file and select "Check out file" to download it (794.6 MB zipped, 2.75 GB unzipped).

This is the filter used by the ilastik program to detect cells. Move this file to the "filters" folder in this directory.

In [4]:
# make sure there are no errors
assert glob.glob('filters/*.ilp'), "Please move your downloaded ilastik .ilp file into the filters folder!"

> NOTE: the .ilp file is a filter created using Ilastik, a software for image segmentation.  In our example, it is used to detect cells based on pixel segmentation.
>
>Should you plan to use Ilastik for the same purpose (ie. creating filters for cell segmentation), it is required that you download Ilastik 1.3.3 for the successful running of the BrainQuant3D code.

## 5. Set up configuration file
Now you need to set up the configuration file. This file will tell BrainQuant3D where to place temp data and where to find the atlas data. The configuration template file, default.conf, can be found in the package install location.

Temp_path: This is the location where temp data will be stored. It should be on a high-performance SSD. Preferably an NVMe. If the path does not exist, it will be created.

Rigid_default, Affine_default, BSpline_default, Labeled_default, Annotations_default: These are the paths to the Allen Brain Atlas datasets downloaded earlier.

Processing_cores, Thread_ram_max_Gb: These parameters specify how many processing cores to use and how much RAM to user per core. For example, if your system has 10 cores and 256 GB of RAM, you would set "Processing_cores" to 10 and "Thread_ram_max_Gb" to a value such as 22. This would use 10 x 22 = 220 GB of total RAM. It's a good idea to leave a little available RAM so as to not overload the system.

Adjust variables as necessary below:

In [ ]:
### locate temporary directory path on your computer
!echo "temporary directory:" $TMPDIR

### uncomment to find the number of cores and amount of RAM supported by your computer (linux)
# !lscpu | egrep 'Model name|Socket|Thread|NUMA|CPU\(s\)' #look for number of CPUs
# !free -h #max RAM

### for windows:
# !wmic cpu get caption, deviceid, name, numberofcores, maxclockspeed, status #look for number of cores
# !systeminfo #look for max RAM

### for mac:
!sysctl -a | grep hw.ncpu #number of cores
!sysctl -n hw.memsize #max RAM in bytes; divide by (1024**3) for GB
print(8589934592/(1024**3), "GB RAM")

In [ ]:
### replace temp_directory variable with above output (optional). For example, on mac computers, it might look like this: temp_directory = '/var/folders/47/6bm96nxd0ks5x370bsvkpghh0000gn/T/'. 
#If the temporary directory cannot be automatically found by the command above, we will create a temporary directory in the next step.

temp_directory = "/var/folders/47/6bm96nxd0ks5x370bsvkpghh0000gn/T/"
# temp_directory = '' #!!!NOTE: change this accordingly

In [ ]:

#if there is no temp_directory provided, create a /tmp folder in the root directory of the computer
if temp_directory == '':
    temp_directory = Path("/tmp")
    temp_directory.mkdir(parents=True, exist_ok=True)

processing_cores = 1 #!!!NOTE: change this accordingly
thread_ram_max_GB = 4 #!!!NOTE: change this accordingly

#find installation address of bq3d package
bq3d_package_directory = Path(bq3d.__file__).parent
assert Path(bq3d_package_directory, "default.conf").exists(), f"cannot find BrainQuant3D configuration file in the directory {bq3d_package_directory}"
print("BQ3D_PACKAGE_DIRECTORY:", bq3d_package_directory)

***UNCOMMENT the "Ilastik_path" as necessary (line with !!! comment)***

Run the following code blocks to automatically create a configuration file:

In [ ]:
### create updated config file
try:
    parigid_path = Path(brain_atlas_directory, "ParRigid.txt")
    paraffine_path = Path(brain_atlas_directory, "ParAffine.txt")
    parbspline_path = Path(brain_atlas_directory, "ParBSpline.txt")
    annotation_path = Path(brain_atlas_directory, "ARA2", "annotation_25_right.tif")
    collapse_path = Path(brain_atlas_directory, "ARA2_annotation_info_collapse.csv")
except Exception as e: print(e, f"file does not exist in the provided directory {brain_atlas_directory}")

config_data = {
    'user': {
        'default': {
            #paths
            'Ilastik_path': f'{str(bq3d_package_directory)}/.external/ilastik-1.3.3/Contents/ilastik-release', #!!!NOTE: for mac
            # 'Ilastik_path': f'{str(bq3d_package_directory)}/.external/ilastik-1.3.3', #!!!NOTE: for linux and windows
            'Elastix_path': f'{str(bq3d_package_directory)}/.external/elastix-5.0.0',
            'Temp_path':    temp_directory, 

            #brain atlas
            'Rigid_default':       f'{str(parigid_path)}',
            'Affine_default':      f'{str(paraffine_path)}',
            'BSpline_default':     f'{str(parbspline_path)}',
            'Labeled_default':     f'{str(annotation_path)}',
            'Annotations_default': f'{str(collapse_path)}',
            'Console_level':       'verbose',

            #processing
            'Processing_cores':    processing_cores, 
            'Thread_ram_max_Gb':   thread_ram_max_GB, 
        }
    }
}
pprint(config_data)

### save to the bq3d folder
with open(Path(bq3d_package_directory,"brainquant3d.conf"), mode="w+") as f:
    yaml.dump(config_data, f, sort_keys=False, indent=4, allow_unicode=True)

## 6. Ensure configuration files are properly imported.
Try to import the bq3d package-- there shouldn't be any warnings that pop up now that the configuration file is set.

In [ ]:
import bq3d

You can now move onto the tutorial Jupyter Notebook (01_bq3d_tutorial.ipynb).